In [2]:
import pandas as pd 

In [3]:
earthquakes = pd.read_csv('earthquake_merged2.csv')
# earthquakes.columns

In [4]:
earthquakes = earthquakes[earthquakes["Year"] > 1999]
# earthquakes

In [5]:
country_counts = earthquakes.groupby("Country Name").size().reset_index(name="Count")
country_counts = country_counts.sort_values(by="Count", ascending=False)
country_counts

,Country Name,Count
17,CHINA,155
46,INDONESIA,140
47,IRAN,86
49,JAPAN,80
103,UNITED STATES,58
...,...,...
38,GULF OF MEXICO,1
51,KAZAKHSTAN,1
50,JORDAN,1
86,SLOVENIA,1


In [6]:
# Group by Country Name to get avg magnitude
avg_magnitude = earthquakes.groupby("Country Name")["Mag"].mean().reset_index(name="Average Magnitude")

# Merge the average magnitude with the country_counts df
country_counts = country_counts.merge(avg_magnitude, on="Country Name", how="left")

# country_counts

In [7]:
import plotly.express as px

fig = px.treemap(
    country_counts,
    path=["Country Name"],
    values="Count",
    title="Earthquake Counts and Average Magnitudes Across Countries (2000–2024)",
    hover_data={"Count": True, "Average Magnitude": True}  # Pass 'Average Magnitude' explicitly
)

# custom hover 
fig.update_traces(
    customdata=country_counts[["Average Magnitude"]].values,  # Include custom data for hovertemplate
    hovertemplate="<b>%{label}</b><br>Count: %{value}<br>Average Magnitude: %{customdata[0]:.2f}<extra></extra>"
)

fig.show()


## write-up for tree map: 
The treemap highlights global earthquake occurrences (2000–2024) by country, with box size representing the total number of earthquakes and hover data showing average magnitudes. Countries like China, Indonesia, and Japan dominate with frequent seismic activity, while smaller boxes, such as Panama and Haiti, indicate fewer recorded events. High-frequency countries like China and Indonesia experience frequent but varied-magnitude earthquakes, whereas nations like Japan and Chile endure fewer but more intense events. This visualization underscores the global disparities in earthquake frequency and severity, emphasizing the need for tailored disaster preparedness strategies based on regional seismic activity patterns.

In [8]:
file_path = "historicalCPI.xlsx"  

cpi_data = pd.read_excel(file_path, skiprows=10)
# Make the first row the new header
cpi_data.columns = cpi_data.iloc[0]  # Set the first row as column headers
df = cpi_data[1:]  # Drop the first row from the data

# Reset index for cleaner df
df = df.reset_index(drop=True)

# only will be using december of every year (3rd to last col)for inflation adjustment 
df = df[["Year", "Dec"]]  # Use the explicit column name for December CPI
df.columns = ["Year", "CPI"]  # Rename for simplicity

# only 2000s and on 
df = df[df["Year"].astype(int) >= 2000]
# Add 315.664 to the CPI value for the year 2024 (missing)
df.loc[df["Year"] == 2024, "CPI"] = df.loc[df["Year"] == 2024, "CPI"].fillna(0) + 315.664

    # Display the up

# cpi_data
# df


/Users/lauraboelsterli/anaconda3/envs/ds4200/lib/python3.11/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning:

Workbook contains no default style, apply openpyxl's default

/var/folders/7l/h7qnw85j2hlf4tgr241lw5f00000gn/T/ipykernel_92272/2874546429.py:18: FutureWarning:

Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`



In [9]:
# Group by Country and Year, calculating total counts and sum of damages
# Include average magnitude in the aggregation
bubble_data = earthquakes.groupby(["Country Name", "Year"]).agg(
    Earthquake_Count=("Country Name", "count"),
    Total_Damage_Mil=("Total Damage ($Mil)", "sum"),
    Average_Magnitude=("Mag", "mean")
).reset_index()

# Filter out rows where Total_Damage_Mil is NaN or 0
bubble_data = bubble_data[bubble_data["Total_Damage_Mil"] > 0]
bubble_data["Average_Magnitude"] = bubble_data["Average_Magnitude"].round(2)

In [10]:
# Adjust bubble chart data with inflation-adjusted damages using CPI

# Merging bubble_data with the CPI dataset
adjusted_bubble_data = bubble_data.merge(df, on="Year", how="left")

# Add current CPI aka 2024
current_cpi = df.loc[df["Year"] == 2024, "CPI"].values[0]

# calculate inflation-adjusted damages
adjusted_bubble_data["Inflation_Adjusted_Damage_Mil"] = (
    adjusted_bubble_data["Total_Damage_Mil"]
    * (current_cpi / adjusted_bubble_data["CPI"])
)
adjusted_bubble_data["Inflation_Adjusted_Damage_Mil"] = adjusted_bubble_data["Inflation_Adjusted_Damage_Mil"].fillna(0)


# roudning the adjusted damages 
adjusted_bubble_data["Inflation_Adjusted_Damage_Mil"] = adjusted_bubble_data["Inflation_Adjusted_Damage_Mil"].round(2)

# adjusted_bubble_data


/var/folders/7l/h7qnw85j2hlf4tgr241lw5f00000gn/T/ipykernel_92272/1727538281.py:14: FutureWarning:

Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`



In [11]:
# Count the number of countries omitted from the graph for later write up about the viz and dataset 
# all countries in the original dataset
all_countries = earthquakes["Country Name"].unique()
print("all countries counted is", len(all_countries))
# get the countries included in the bubble_data after filtering
included_countries = adjusted_bubble_data["Country Name"].unique()
# Find omitted countries
omitted_countries = set(all_countries) - set(included_countries)
num_omitted_countries = len(omitted_countries)
# count of omitted countries
print(f"Number of countries omitted from the graph: {num_omitted_countries}")

# 47 out of 109 countries represented in that graph

all countries counted is 109
Number of countries omitted from the graph: 62


In [17]:
# # Create the bubble chart with size scaling
fig = px.scatter(
    adjusted_bubble_data,
    x="Year",
    y="Earthquake_Count",
    size="Inflation_Adjusted_Damage_Mil",
    color="Country Name",
    
    title="Earthquake Occurrences and Economic Impacts in Total Damages (2000–2024)",
    labels={
        "Earthquake_Count": "Earthquake Counts",
        "Inflation_Adjusted_Damage_Mil": "Total Damage ($Mil)",
        "Average_Magnitude": "Average Magnitude"
    },
    hover_data={"Inflation_Adjusted_Damage_Mil": True, "Country Name": True,"Average_Magnitude": True},
    size_max=80
)

# Improve layout
fig.update_layout(
    xaxis_title="Year",
    yaxis_title="Earthquake Count",
    legend_title="Country",
    showlegend=True  # Ensure legend for countries is displayed
)


# Optionally, add a note about bubble size
fig.add_annotation(
    xref="paper", yref="paper",
    x=1.0, y=1.1,
    text="*Bubble size shows total damage in Millions ($M, adjusted to 2024 CPI)",
    showarrow=False,
    font=dict(size=12),
    align="left"
)

fig.show()


## write-up for bubble chart: 
The bubble chart highlights global earthquake trends (2000–2024) by showing earthquake counts (y-axis), years (x-axis), and inflation-adjusted damages (bubble size) across countries. Larger bubbles, such as those for China (2008), Chile (2010), and Japan (2011), indicate catastrophic events with significant economic impacts, including infrastructure destruction and secondary effects like tsunamis. Countries like Indonesia show high earthquake counts but smaller damages, reflecting frequent lower-magnitude events. The chart reveals that high-frequency earthquakes don't always result in high economic losses, emphasizing the role of disaster preparedness and infrastructure resilience. Clusters in the early 2010s suggest major global seismic activity, while smaller bubbles for countries like Albania or Costa Rica highlight localized, less costly events. This analysis underscores the economic disparities in how countries are affected by and recover from seismic disasters.

Notes for me:
Consumer Price Index for All Urban Consumers (CPI-U),
https://data.bls.gov/timeseries/CUUR0000SA0?years_option=all_years us buereu of statistics 

count per year and its avg magnitude that year in that given country. total money damages are calculated consider and have been converted to current monetary value:
Infrastructure Damage:
Roads, bridges, power lines, water systems, and other public infrastructure that require repair or rebuilding.
Costs associated with restoring services like electricity, water, and transportation.
Economic Losses:
Direct and indirect economic impacts, such as:
Business interruptions due to damaged facilities or power outages.
Agricultural losses if farmlands are affected.
Losses from disrupted trade or tourism.
Loss of Personal Assets:
Damage to personal belongings, vehicles, and other private property.

the inflation-adjusted damages for earthquakes, tells us a story of economic impacts of seismic events across different countries and years, normalized to today's dollar value (2024)

we can see 2011 which is when Tōhoku earthquake and tsunami biggest impact on 
#maybe also some tsunami stuff?? for future reserxh the correlation of that and the imact on the damages etc / magnitude/ popualtion / deaths 


idea:
total deaths and population  and then seperately as points put the plots of earthquake occurences???

## dataset notes:
-merged with population, which i got from the world bank group https://databank.worldbank.org/Population-and-GDP-by-Country/id/29c4df41# 
-i converted the total damage in $ to current monetary value with the consumer price index 
-lots of null values had to clean up and filter out columns that were to large (over 3500)
-counted null values in viz where necessary and renamed the coutry anmes into a standarduzed format as sometimes were regions were mentioned 


-from the dataset web: 
damage explaination of dolalrs from earthquakes and after effects 
The dollar value listed is the value at the time of the event. 
To convert the damage to current dollar values, please use the Consumer 
Price Index Calculator. Monetary conversion tables for the time of the 
event were used to convert foreign currency to U.S. dollars.
The dollar amount reflects the value at the time of the event, which could 
be decades ago and thus not reflective of current monetary value
Consumer Price Index (CPI) Calculator to adjust to current moentary value 


-cpi adjsutment 
-country names
-null values 
-data starting after 2000s
